<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-17 12:09:58
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.33 C
CY Investment:  1.60 C
Reserve:  4.98 L
Current:  1.43 C
-------------------
Today PnL: 93.60 K (0.66%)
Current PnL: -31.43 L (-19.64%)
CY Booked + Current PnL: -15.75 L (-9.84%)
-------------------
Total profit:  1.21 L
Total loss:  -32.63 L
-------------------
Total Booked + Current PnL: 11.23 L (8.45%)
Total Booked PnL: 42.66 L (32.07%)
Curr Year Booked PnL: 15.68 L (10.96%)
Prev Year Booked PnL: 26.98 L (20.29%)
Est FTT:  2.40 C
Est FTT PnL: 96.79 L (67.68%)
Deployed:  1.33 C
Current:  1.43 C
CAGR/XIRR %: 3.77%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.34,3.72,17.21,21.57,17490.0,3645.0,101625.0,397.04,-10.57,61.0,H-SC,4.33,167.0,0.21,0.74,48.30,XR,ATH,FINANCE
18,COALINDIA,-0.63,3.25,15.48,19.23,24047.0,4884.0,155344.0,484.83,24.82,48.0,L-LC,11.24,185.0,0.20,1.13,23.16,XY25,ATH,MINING
77,TTKPRESTIG,0.68,-26.95,37.01,0.09,27248.0,-27155.0,73622.0,770.00,70.97,41.0,M-SC,9.09,250.0,-1.00,0.53,1.33,OX40N,NTT,DURABLES
36,ICICIGI,0.47,2.29,16.67,19.34,34121.0,4577.0,204686.0,2252.93,-13.95,60.0,X-MC,3.02,66.0,0.13,1.48,18.59,X40,ATH,INSURANCE
51,MEDANTA,1.04,-3.39,28.44,24.08,34879.0,-4308.0,122642.0,1486.00,-4.09,55.0,X-SC,11.47,87.0,-0.12,0.89,12.33,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.19,-47.90,179.66,45.70,165496.0,-84699.0,92116.0,26.4,-4.45,81.0,M-SC,24.63,204.0,-0.51,0.67,51.77,XY24,NTT,TRAVEL
42,INFY,4.31,-3.03,38.44,34.24,116626.0,-9491.0,303397.0,1972.0,-23.80,33.0,X-LC,4.30,3.0,-0.08,2.20,5.00,X40,NTT,IT
72,TATAELXSI,3.40,-28.16,83.77,32.03,79343.0,-37120.0,94715.0,9161.0,-10.11,40.0,H-SC,6.81,158.0,-0.47,0.69,6.09,OX40N,NTT,IT
64,SHALBY,3.37,-38.79,104.38,25.10,115737.0,-70263.0,110880.0,327.0,714.99,37.0,M-SC,17.17,236.0,-0.61,0.80,3.37,XY24,NTT,HEALTHCARE
69,SURYODAY,3.19,-27.51,69.73,23.03,90513.0,-49266.0,129805.0,216.0,42.54,37.0,H-SC,2.49,166.0,-0.54,0.94,28.79,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIS,-1.94,-29.60,72.24,21.26,56470.0,-32862.0,78170.0,528.00,1836.79,34.0,H-SC,4.20,168.0,-0.58,0.57,7.77,OX40N,NTT,MISC
55,RAJESHEXPO,-1.79,-67.93,212.05,0.07,93281.0,-93187.0,43990.0,518.00,1512.37,43.0,L-SC,9.01,268.0,-1.00,0.32,9.40,OX40N,NTT,JEWELLERY
45,JCHAC,-1.55,-39.46,65.16,-0.01,50418.0,-50430.0,77375.0,2282.00,15726.09,48.0,M-SC,17.14,231.0,-1.00,0.56,7.27,OX40N,BTT,AC
65,SIEMENS,-1.54,-14.93,47.55,25.51,75274.0,-27790.0,158305.0,4671.50,32.94,54.0,H-LC,1.97,51.0,-0.37,1.15,16.66,AR,ATH,ELECTRICAL
50,MASFIN,-1.34,3.72,17.21,21.57,17490.0,3645.0,101625.0,397.04,-10.57,61.0,H-SC,4.33,167.0,0.21,0.74,48.30,XR,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1.05,2.96,101.94,107.91,154961.0,4368.0,152012.0,1641.55,-7.15,64.0,M-SC,9.50,220.0,0.03,1.10,14.36,OX40N,ATH,CABLES
77,TTKPRESTIG,0.68,-26.95,37.01,0.09,27248.0,-27155.0,73622.0,770.00,70.97,41.0,M-SC,9.09,250.0,-1.00,0.53,1.33,OX40N,NTT,DURABLES
72,TATAELXSI,3.40,-28.16,83.77,32.03,79343.0,-37120.0,94715.0,9161.00,-10.11,40.0,H-SC,6.81,158.0,-0.47,0.69,6.09,OX40N,NTT,IT
66,SIS,-1.94,-29.60,72.24,21.26,56470.0,-32862.0,78170.0,528.00,1836.79,34.0,H-SC,4.20,168.0,-0.58,0.57,7.77,OX40N,NTT,MISC
40,INDIGOPNTS,0.07,-29.81,42.50,0.02,52073.0,-52035.0,122524.0,1408.00,86.94,32.0,M-SC,8.32,222.0,-1.00,0.89,6.58,OX40N,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.63,3.25,15.48,19.23,24047.0,4884.0,155344.0,484.83,24.82,48.0,L-LC,11.24,185.0,0.20,1.13,23.16,XY25,ATH,MINING
26,FINCABLES,1.05,2.96,101.94,107.91,154961.0,4368.0,152012.0,1641.55,-7.15,64.0,M-SC,9.50,220.0,0.03,1.10,14.36,OX40N,ATH,CABLES
50,MASFIN,-1.34,3.72,17.21,21.57,17490.0,3645.0,101625.0,397.04,-10.57,61.0,H-SC,4.33,167.0,0.21,0.74,48.30,XR,ATH,FINANCE
1,ABB,-0.70,7.28,35.69,45.57,100166.0,19037.0,280656.0,7934.00,-31.10,69.0,H-MC,3.53,125.0,0.19,2.03,24.63,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,MASFIN,-1.34,3.72,17.21,21.57,17490.0,3645.0,101625.0,397.04,-10.57,61.0,H-SC,4.33,167.0,0.21,0.74,48.30,XR,ATH,FINANCE
1,ABB,-0.70,7.28,35.69,45.57,100166.0,19037.0,280656.0,7934.00,-31.10,69.0,H-MC,3.53,125.0,0.19,2.03,24.63,AR,NTT,ELECTRICAL
26,FINCABLES,1.05,2.96,101.94,107.91,154961.0,4368.0,152012.0,1641.55,-7.15,64.0,M-SC,9.50,220.0,0.03,1.10,14.36,OX40N,ATH,CABLES
39,INDIAMART,0.88,-4.37,116.18,106.72,137024.0,-5395.0,117941.0,4810.62,-56.84,51.0,H-SC,7.72,140.0,-0.04,0.85,17.98,AR,ATH,MISC
85,ZYDUSLIFE,-0.32,-4.30,41.72,35.63,83677.0,-9005.0,200569.0,1286.17,-16.67,52.0,H-MC,5.60,120.0,-0.11,1.45,11.66,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,0.07,-33.87,137.69,57.19,296971.0,-110446.0,215681.0,485.00,-62.20,31.0,X-SC,17.42,81.0,-0.37,1.56,1.23,XY24,NTT,FMCG
73,TCS,1.61,-24.38,56.78,18.55,162396.0,-92227.0,286010.0,4311.58,-31.50,31.0,X-LC,3.63,2.0,-0.57,2.07,2.15,X40,ATH,IT
13,BERGEPAINT,0.31,-17.37,46.60,21.14,87542.0,-39480.0,187859.0,680.00,-19.19,32.0,X-MC,11.93,68.0,-0.45,1.36,1.17,XY24,NTT,PAINTS
42,INFY,4.31,-3.03,38.44,34.24,116626.0,-9491.0,303397.0,1972.00,-23.80,33.0,X-LC,4.30,3.0,-0.08,2.20,5.00,X40,NTT,IT
53,PGHH,0.07,-13.24,52.16,32.01,96625.0,-28272.0,185248.0,17616.87,-41.36,34.0,X-MC,9.32,57.0,-0.29,1.34,0.13,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,DABUR,0.44,0.98,42.30,43.70,105962.0,2434.0,250502.0,735.00,-8.39,54.0,X-MC,1.83,70.0,0.02,1.81,16.18,XY24,BTT,FMCG
65,SIEMENS,-1.54,-14.93,47.55,25.51,75274.0,-27790.0,158305.0,4671.50,32.94,54.0,H-LC,1.97,51.0,-0.37,1.15,16.66,AR,ATH,ELECTRICAL
78,UNITDSPR,0.42,3.35,16.85,20.77,40775.0,7843.0,241987.0,1644.00,-8.32,60.0,X-MC,1.99,64.0,0.19,1.75,11.00,X40N,NTT,BREWERIES
34,HINDUNILVR,-0.46,-10.20,26.59,13.67,61378.0,-26233.0,230830.0,2922.00,-33.31,40.0,X-LC,2.87,9.0,-0.43,1.67,8.97,XY25,NTT,FMCG
36,ICICIGI,0.47,2.29,16.67,19.34,34121.0,4577.0,204686.0,2252.93,-13.95,60.0,X-MC,3.02,66.0,0.13,1.48,18.59,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,0.13,-31.31,138.99,64.15,227165.0,-74511.0,163440.0,3906.00,-48.05,38.0,X-SC,8.38,85.0,-0.33,1.18,0.13,XY24,BTT,CEMENT
53,PGHH,0.07,-13.24,52.16,32.01,96625.0,-28272.0,185248.0,17616.87,-41.36,34.0,X-MC,9.32,57.0,-0.29,1.34,0.13,X40,ATH,FMCG
2,ABBOTINDIA,0.53,-11.34,33.16,18.07,52585.0,-20275.0,158580.0,35195.00,-24.65,36.0,X-MC,10.49,58.0,-0.39,1.15,0.53,X40,ATH,PHARMA
12,BATAINDIA,1.00,-46.00,150.37,35.20,104482.0,-59187.0,69483.0,2096.00,-8.88,37.0,X-SC,21.43,91.0,-0.57,0.50,1.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,0.08,-23.85,104.16,55.47,192975.0,-58018.0,185268.0,181.50,-34.73,38.0,X-MC,17.38,60.0,-0.30,1.34,1.00,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,-0.22,-30.40,105.23,42.84,47611.0,-19761.0,45245.0,424.00,-54.27,50.0,X-SC,19.45,80.0,-0.42,0.33,8.83,XY24,NTT,MISC
58,RELAXO,2.09,-53.09,224.68,52.31,152994.0,-77066.0,68094.0,1176.00,-50.05,40.0,X-SC,14.19,92.0,-0.50,0.49,2.09,X40N,NTT,FOOTWEAR
12,BATAINDIA,1.00,-46.00,150.37,35.20,104482.0,-59187.0,69483.0,2096.00,-8.88,37.0,X-SC,21.43,91.0,-0.57,0.50,1.00,X40,NTT,FOOTWEAR
51,MEDANTA,1.04,-3.39,28.44,24.08,34879.0,-4308.0,122642.0,1486.00,-4.09,55.0,X-SC,11.47,87.0,-0.12,0.89,12.33,XY24,NTT,HEALTHCARE
35,HONAUT,0.53,-22.98,86.09,43.32,107991.0,-37432.0,125440.0,58357.33,-32.37,39.0,X-SC,10.92,90.0,-0.35,0.91,1.85,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,2.48,-20.14,38.71,10.78,74295.0,-48392.0,191926.0,452.00,-53.09,50.0,X-LC,18.81,1.0,-0.65,1.39,5.71,X40,NTT,FMCG
73,TCS,1.61,-24.38,56.78,18.55,162396.0,-92227.0,286010.0,4311.58,-31.50,31.0,X-LC,3.63,2.0,-0.57,2.07,2.15,X40,ATH,IT
42,INFY,4.31,-3.03,38.44,34.24,116626.0,-9491.0,303397.0,1972.00,-23.80,33.0,X-LC,4.30,3.0,-0.08,2.20,5.00,X40,NTT,IT
75,TMPV,0.82,-22.04,57.75,22.98,101699.0,-49796.0,176102.0,600.00,-82.53,62.0,X-LC,3.12,4.0,-0.49,1.28,12.58,XY24,NTT,AUTO
33,HCLTECH,2.88,-2.40,26.88,23.83,63467.0,-5812.0,236112.0,1908.19,2.56,34.0,X-LC,4.84,8.0,-0.09,1.71,13.28,X40,ATH,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1.33,-30.28,90.62,32.89,48776.0,-23382.0,53825.0,1800.00,-353.50,65.0,L-MC,10.90,258.0,-0.48,0.39,48.23,XR,NTT,BANKS
67,SONACOMS,0.77,-8.46,51.95,39.09,48106.0,-8558.0,92601.0,804.02,-27.49,63.0,M-MC,6.00,193.0,-0.18,0.67,31.00,AR,ATH,AUTO
50,MASFIN,-1.34,3.72,17.21,21.57,17490.0,3645.0,101625.0,397.04,-10.57,61.0,H-SC,4.33,167.0,0.21,0.74,48.30,XR,ATH,FINANCE
79,VAIBHAVGBL,2.07,-22.47,107.37,60.77,152142.0,-41076.0,141699.0,521.00,63.64,58.0,H-SC,0.76,160.0,-0.27,1.03,28.47,XR,NTT,JEWELLERY
18,COALINDIA,-0.63,3.25,15.48,19.23,24047.0,4884.0,155344.0,484.83,24.82,48.0,L-LC,11.24,185.0,0.20,1.13,23.16,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,EASEMYTRIP,19.19,-47.90,179.66,45.70,165496.0,-84699.0,92116.0,26.40,-4.45,81.0,M-SC,24.63,204.0,-0.51,0.67,51.77,XY24,NTT,TRAVEL
1,ABB,-0.70,7.28,35.69,45.57,100166.0,19037.0,280656.0,7934.00,-31.10,69.0,H-MC,3.53,125.0,0.19,2.03,24.63,AR,NTT,ELECTRICAL
79,VAIBHAVGBL,2.07,-22.47,107.37,60.77,152142.0,-41076.0,141699.0,521.00,63.64,58.0,H-SC,0.76,160.0,-0.27,1.03,28.47,XR,NTT,JEWELLERY
67,SONACOMS,0.77,-8.46,51.95,39.09,48106.0,-8558.0,92601.0,804.02,-27.49,63.0,M-MC,6.00,193.0,-0.18,0.67,31.00,AR,ATH,AUTO
11,BANDHANBNK,0.36,-18.93,137.35,92.42,310178.0,-52729.0,225830.0,400.00,93.97,69.0,H-SC,7.48,174.0,-0.17,1.64,30.51,XY24,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.26
1,25,44.60
2,50,76.64


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    46.31
MC    30.27
LC    23.40
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.78
X40      23.30
X40N     15.13
AR        9.03
XR        8.79
XY25      8.74
OX40N     7.32
SR        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    22.85
H-SC    22.71
X-LC    20.02
X-SC    11.80
M-SC    11.01
H-MC     4.94
M-MC     2.09
H-LC     1.15
L-LC     1.13
M-LC     1.10
L-SC     0.79
L-MC     0.39
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.00,-14.18,52.55
FINANCE,13.31,-13.64,58.36
IT,11.76,-31.78,97.54
MISC,7.07,-33.14,87.60
ELECTRICAL,6.27,-8.41,47.96
PAINTS,5.13,-30.58,49.98
INSURANCE,4.85,-1.27,35.93
PHARMA,4.10,-3.84,36.53
AUTO,3.19,-19.28,60.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3274139.0,21
AR,1324462.0,10
X40,1279299.0,15
XR,1278323.0,12
X40N,1030560.0,10
OX40N,783159.0,10
XY25,412156.0,6
SR,296847.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3660405.0,24
M-SC,1397686.0,14
X-MC,1356728.0,15
X-SC,1173215.0,10
X-LC,1127503.0,12
H-MC,395549.0,3
L-SC,182083.0,2
M-LC,140578.0,1
M-MC,97101.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1251063.0      6
           AR         943365.0      5
           XR         822370.0      7
M-SC       XY24       802897.0      6
X-SC       XY24       606626.0      4
X-LC       X40        562906.0      6
X-MC       X40        542420.0      7
           X40N       411169.0      4
X-SC       X40N       392616.0      4
H-SC       OX40N      346760.0      4
M-SC       OX40N      343118.0      5
H-SC       SR         296847.0      2
X-LC       X40N       226775.0      2
H-MC       XY24       211706.0      1
X-MC       XY25       209635.0      2
X-LC       XY24       208343.0      2
X-MC       XY24       193504.0      2
H-MC       AR         183843.0      2
M-SC       XR         177797.0      2
X-SC       X40        173973.0      2
M-LC       XR         140578.0      1
X-LC       XY25       129479.0      2
L-SC       OX40N       93281.0      1
           XR          88802.0      1
H-LC       AR          75274.0      1
M-SC       AR          73874.0      1
M-MC       XY25        48995.0      1
L-MC       XR          48776.0      1
M-MC       AR          48106.0      1
L-LC       XY25        24047.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
